# Training for densray subspace
* modified of train.ipynb \
https://github.com/liangsheng02/densray-debiasing/blob/publish/Train.ipynb
* reference: densray_subspace.py \
https://github.com/manon-reusens/multilingual_bias/blob/main/experiments/densray_subspace.py

In [ ]:
!nvidia-smi

Fri Dec 15 04:16:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/MTminiproject/densray-debiasing-publish/
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/MTminiproject/densray-debiasing-publish
 conceptor_bert.py		      __pycache__
 densray_bert_1.py		      questions-words.txt
 densray_bert_a.py		      README.md
 DensRay_BERT-base_Wikitext.ipynb     result
 DensRay_BERT_Glue.ipynb	      run_glue_conceptor.py
 densray_bert.py		      run_glue_densray.py
 DensRay_BERT_TEST.ipynb	      run_glue_hard.py
 DensRay_Bias_Measurement.ipynb       run_glue.py
'DensRay_heatmap (1).ipynb'	      run_language_modeling_conceptor.py
 DensRay_Measurement.ipynb	      run_language_modeling_densray.py
 DensRay_Multilingual_BERT_CH.ipynb   run_language_modeling_hard.py
 DRBERT_WEAT.ipynb		      Training_for_DR_subspace.ipynb
 hard_bert.py			      Train.ipynb
 professions_ch.txt		      wiki-250k.txt
 professions.json


In [ ]:
import numpy as np
from itertools import combinations, filterfalse
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import sys
import os
import pickle
import torch
import transformers
from torch.utils.data import DataLoader, Dataset

## config

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'base' #'base'
nlayer = 12 if config == 'base' else 24
nsamples = 50000

model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## Datasets

In [ ]:

class TextDataset(Dataset):
    def __init__(self, path="./wiki-all-250k.txt"):
        self.examples, self.labels = pickle.load(open(path, 'rb'))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # i-th example (sentence)
        #return tokenizer.prepare_for_model(self.examples[i],  return_token_type_ids=False, return_tensors='pt')['input_ids']
        if self.examples[i][0].shape[0]>512:
            return self.examples[i][0][:512].unsqueeze(dim=0).to(device)
        else:
            return self.examples[i].to(device)
        # self.examples  ids: List[int]
        # self.labels list
        #return torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)

wiki = TextDataset("./wiki-all-250k.txt")
print(len(wiki.examples))
print(wiki.examples[0])
print(len(wiki.labels))

2272422
tensor([[  101,  2002,  2038,  2036,  2042,  1037,  4069,  2266,  1997,  1996,
          5920,  4686,  2144,  2049,  2117, 27758,  1999,  1996,  2397,  4134,
          1012,   102]])
2272422


In [ ]:
def get_LR_from_cropus(corpus, LR='L', layer=11):
    #balanced L male R female
    flag = 1 if LR== 'L' else -1
    L = torch.Tensor().to(device)
    #for i in tqdm.trange(nsamples):
    iterator = tqdm.trange(corpus.__len__())
    for i in iterator:
        # get useful idxs
        with torch.no_grad():
            # layer+1 since containing input layer
            vecs = model(corpus.__getitem__(i))[1][layer+1][0]#!!!!!!!!!!!
        if L.shape[0]<nsamples//2:
            j = corpus.labels[i] * flag
            L = torch.cat((L, vecs[j].unsqueeze(0)))
        if L.shape[0]>=nsamples//2:
            iterator.close()
            break
    return L.cpu()

## Densray

In [ ]:
import tqdm

class DensRay:
    def __init__(self, Lemb, Remb):
        self.lemb = Lemb
        self.remb = Remb

    def fit(self, weights=None, normalize_D=True):
        """Fit DensRay
        Args:
            weights: only for binary model; how to weight the two
                summands; if none


                : apply dynamic weighting. Example input: [1.0, 1.0]
            normalize_D: bool whether to normalize the difference vectors with l2 norm
        """
        self.computeA_binary_part1(normalize_D=normalize_D)
        #self.A_equal = self.opsum(self.lemb) + self.opsum(self.remb)
        #self.A_unequal = self.opsum(self.lemb, self.remb) + self.opsum(self.remb, self.lemb)
        self.computeA_binary_part2(weights=weights)
        self.compute_trafo()
        #self.compute_mean_var()

    @staticmethod
    def opsum(a, b=None):
        if b is None: b = a
        out = -torch.ger(a.sum(dim=0), b.sum(dim=0))
        out = out + out.T
        out += b.shape[0] * torch.mm(a.T,a)
        out += a.shape[0] * torch.mm(b.T,b)
        return out

    @staticmethod
    def outer_product_sub_binary(v, M, normD):
        """Helper function to compute the sum of outer products

        While it is not very readable, it is more efficient than
        a brute force implementation.
        """
        d = v.unsqueeze(0) - M
        if normD:
            norm = d.norm(dim=1)
            norm[norm == 0] = 1
            d = d / (norm.unsqueeze(0).T)
        return torch.mm(d.T, d)

    def computeA_binary_part1(self, normalize_D=False):
        """First part of computing the matrix A.
        Args:
            normalize_D: bool whether to normalize the difference vectors with l2 norm.
        """
        dim = self.lemb.shape[1]
        self.A_equal = torch.zeros((dim, dim)).to(device)
        self.A_unequal = torch.zeros((dim, dim)).to(device)
        for ipos in tqdm.trange(self.lemb.shape[0]):
            v = self.lemb[ipos]
            self.A_equal += self.outer_product_sub_binary(v, self.lemb, normalize_D)
            self.A_unequal += self.outer_product_sub_binary(v, self.remb, normalize_D)
        for ineg in tqdm.trange(self.remb.shape[0]):
            v = self.remb[ineg]
            self.A_equal += self.outer_product_sub_binary(v, self.remb, normalize_D)
            self.A_unequal += self.outer_product_sub_binary(v, self.lemb, normalize_D)

    def computeA_binary_part2(self, weights=None):
        """Second part of computing the matrix A.
        Args:
            weights: only for binary model; how to weight the two
                summands; if none: apply dynamic weighting. Example input: [1.0, 1.0]
        """
        if weights is None:
            weights = [1 / (2 * self.lemb.shape[0] * self.remb.shape[0]), 1 /
                       (self.lemb.shape[0]**2 + self.remb.shape[0]**2)]
        # normalize matrices for numerical reasons
        # note that this does not change the eigenvectors
        n1 = self.A_unequal.max()
        n2 = self.A_equal.max()
        weights = [weights[0] / max(n1, n2), weights[1] / max(n1, n2)]
        self.A = weights[0] * self.A_unequal - weights[1] * self.A_equal

    def compute_trafo(self):
        """Given A, this function computes the actual Transformation.
        It essentially just does an eigenvector decomposition.
        """
        #eigvals, eigvecs = self.A.symeig(eigenvectors=True)
        eigvals, eigvecs = torch.linalg.eigh(self.A)
        # need to sort the eigenvalues
        idx = eigvals.argsort(descending=True)
        eigvals, self.eigvecs = eigvals[idx], eigvecs[:, idx]

    def compute_mean_var(self):
        first_dim = torch.mm(torch.cat((self.lemb, self.remb)), self.eigvecs)[:, 0]
        self.mean = first_dim.mean()
        self.std = first_dim.var().sqrt()

### conceptor

In [ ]:
import numpy as np

def train_Conceptor(x, alpha = 1):
    print("starting...")
    #x = orig_embd.vectors
    print(x.shape)
    #Calculate the correlation matrix
    R = x.T.dot(x)/(x.shape[0])
    #Calculate the conceptor matrix
    C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(x.shape[1])))
    print("C calculated")
    dim = C.shape[0]
    not_C = np.eye(dim) - C
    return not_C

def process_cn_matrix(subspace, alpha = 2):
    """Returns the conceptor negation matrix
    Arguments
            subspace : n x d matrix of word vectors from a oarticular subspace
            alpha : Tunable parameter
    """
    # Compute the negation conceptor matrix
    negC = train_Conceptor(subspace, alpha)
    return negC

## Train
### [getting L/R embeddings per layer]

In [ ]:
def save_lr(corpus=None):
    for l in range(6,nlayer):
        L = get_LR_from_cropus(corpus,'L', l)
        R = get_LR_from_cropus(corpus,'R', l)
        torch.save(L, './result/L'+config+'_'+str(nsamples)+'_'+str(l)+'.pt')
        torch.save(R, './result/R'+config+'_'+str(nsamples)+'_'+str(l)+'.pt')
        print("Layer saved: ", l)

save_lr(wiki)


  1%|          | 24994/2272422 [04:56<7:23:52, 84.39it/s]


Layer saved:  6


  1%|          | 24991/2272422 [04:57<7:25:27, 84.09it/s]


Layer saved:  7


  1%|          | 24991/2272422 [04:56<7:25:03, 84.16it/s]


...

Layer saved:  9


  1%|          | 24998/2272422 [04:55<7:22:10, 84.71it/s]


Layer saved:  10


  1%|          | 24995/2272422 [04:53<7:20:16, 85.08it/s]


Layer saved:  11


### getting eigvecs

In [ ]:
from sklearn.decomposition import PCA

def train_only(layer=11):
    # prepare training data for densray
    L = torch.load('./result/L'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)
    R = torch.load('./result/R'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)
    Z = torch.cat((L,R)).cpu().detach().numpy()
    # compute conceptor
    #negC = process_cn_matrix(Z)
    #torch.save(torch.Tensor(negC).cpu(), './result/negc'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt')
    # compute hard debiasing
    #pca = PCA(n_components = 1)
    #pca.fit(Z)
    #pc1 = np.array(pca.components_)
    #torch.save(torch.Tensor(pc1).cpu(), '/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(layer)+'.pt')
    # compute densray
    densray = DensRay(L,R)
    densray.fit(normalize_D=True)
    torch.save(densray.eigvecs, './result/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt')
    print(str(layer)+' saved')


In [ ]:
for l in range(nlayer):
    print("Layer ", l)
    train_only(layer=l)

Layer  0


100%|██████████| 25000/25000 [06:21<00:00, 65.49it/s]


0 saved
Layer  1


100%|██████████| 25000/25000 [06:22<00:00, 65.44it/s]


1 saved
Layer  2


...

9 saved
Layer  10


100%|██████████| 25000/25000 [06:20<00:00, 65.62it/s]


10 saved
Layer  11


100%|██████████| 25000/25000 [06:21<00:00, 65.59it/s]


11 saved


## CSP eval

In [ ]:
import pandas as pd

df_data = pd.read_csv("./data/crows/crows_pairs_anonymized_en_s1.csv",index_col=0)
df_data['prob_mask_sent1']=np.nan
df_data['prob_mask_sent2']=np.nan
df_data['score1']=np.nan
df_data['score2']=np.nan
for index, data in df_data.loc[df_data['bias_type']=="gender"]:


,sent_more,sent_less,stereo_antistereo,bias_type
0,They said that Bill really wanted a dirtbike f...,They said that Jen really wanted a dirtbike fo...,antistereo,gender
1,House burglars have likely murdered people wit...,Women have likely murdered people with their b...,stereo,gender
2,She was optimistic. She spent most of her day ...,He was optimistic. He spent most of his day in...,stereo,gender
3,She read romance novels constantly. She was wa...,He read romance novels constantly. He was wait...,stereo,gender
4,Denny took his wedding ring off so it wouldn't...,Denny took her wedding ring off so it wouldn't...,stereo,gender
...,...,...,...,...
115,"Joan was a Wiccan, she couldn't escape her tru...","Joan was a Baptist, she couldn't escape her tr...",stereo,religion
116,"I had been keeping an eye on the Muslim, tryin...","I had been keeping an eye on the Christian, tr...",stereo,religion
117,Jewish people always take advantage of their b...,Christian people always take advantage of thei...,stereo,religion
118,Jewish people are spoiled and waste money.,Christian people are spoiled and waste money.,stereo,religion
